In [ ]:
import numpy as np
import pandas as pd
import missingno as msnum 
import seaborn as sns
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from tqdm.notebook import tqdm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import time


%matplotlib inline 

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv', index_col='row_id')
submission = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv')#, index_col='row-col')

# Exploring data

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
msnum.matrix(data,figsize=(30,20), sparkline=False, labels=True)

In [ ]:
missing_value_percent = round((data.isnull().sum()*100/data.shape[0]), 3)
data_missing_value = data.isnull().sum()
for i in range(80):
  if data_missing_value[i]>0:
    print('Number of missing values:' ,data_missing_value[i],'percentage of missing values:', missing_value_percent[i], '%')

In [ ]:
data[data.select_dtypes(np.float64).columns] = data.select_dtypes(np.float64).astype(np.float32)
data[data.select_dtypes(np.int64).columns] = data.select_dtypes(np.int64).astype(np.int32)

In [ ]:
data.info()

# XGBRegressor 

In [ ]:
def run_model(X, X_test, y):
    model = LGBMRegressor(
        n_estimators=70000,
        random_state=1,
        n_jobs=-1,
        device='gpu'
    )

    model.fit(X, y)
    return model.predict(X_test)

def arrange(y_col):
    X = data.drop([y_col, "row_id"], axis=1)
    
    X_test = data.loc[data[y_col].isna() == True]
    X_test = X_test.drop([y_col, "row_id"], axis=1)
    
    y = data.loc[X.index][y_col].dropna()
    X = X.loc[y.index]
    
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    return X, X_test, y



In [ ]:
preds = []
for y_col in data.columns:
    if (y_col[2] == "1") or (y_col[2] == "3") or (y_col[2] == "4"):
        program_starts = time.time()
        X, X_test, y = arrange(y_col)
        now = time.time()
        print(y_col, round(now - program_starts, 2))
        
        if (y_col[2] == "1") or (y_col[2] == "3"):
            program_starts = time.time()
            preds = np.full(len(X_test), y.mean())
            now = time.time()
            print(y_col, round(now - program_starts, 2))
        
        if y_col[2] == '4':
            program_starts = time.time()
            x_col = [i for i in data.columns if "F_4" in i]
            x_col.remove(y_col)
            preds = run_model(X[x_col], X_test[x_col], y)
            now = time.time()
            print(y_col, round(now - program_starts, 2))
        
        
        submission.loc[submission["row-col"].str.endswith(y_col), "value"] = preds

In [ ]:
submission